## PROJECT TITLE

Air Quality & Climate Health Risk Prediction using Machine Learning

## PROBLEM STATEMENT

Air pollution has become a critical climate-related risk, especially in rapidly urbanizing regions of India. Poor air quality leads to respiratory and cardiovascular diseases and, during extreme pollution events (smog, wildfires, stagnant heat), it can escalate into a public health disaster.

## PROJECT STATEMENT

This project uses the Air Quality Data in India (2015–2020) dataset from Kaggle to design a machine learning–based early warning system. It aims to predict air pollution peaks and assess climate-related health risks.

In [7]:
# IMPORTINGG NECESSARY LIBRARIES[WEEK 1]
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [8]:
# LOADING THE DATASET
df = pd.read_csv('city_day.csv')

# DISPLAYING FIRST 10 ROWS
print(df.head(10))

        City        Date  PM2.5  PM10      NO    NO2     NOx  NH3      CO  \
0  Ahmedabad  2015-01-01    NaN   NaN    0.92  18.22   17.15  NaN    0.92   
1  Ahmedabad  2015-01-02    NaN   NaN    0.97  15.69   16.46  NaN    0.97   
2  Ahmedabad  2015-01-03    NaN   NaN   17.40  19.30   29.70  NaN   17.40   
3  Ahmedabad  2015-01-04    NaN   NaN    1.70  18.48   17.97  NaN    1.70   
4  Ahmedabad  2015-01-05    NaN   NaN   22.10  21.42   37.76  NaN   22.10   
5  Ahmedabad  2015-01-06    NaN   NaN   45.41  38.48   81.50  NaN   45.41   
6  Ahmedabad  2015-01-07    NaN   NaN  112.16  40.62  130.77  NaN  112.16   
7  Ahmedabad  2015-01-08    NaN   NaN   80.87  36.74   96.75  NaN   80.87   
8  Ahmedabad  2015-01-09    NaN   NaN   29.16  31.00   48.00  NaN   29.16   
9  Ahmedabad  2015-01-10    NaN   NaN     NaN   7.04    0.00  NaN     NaN   

     SO2      O3  Benzene  Toluene  Xylene  AQI AQI_Bucket  
0  27.64  133.36     0.00     0.02    0.00  NaN        NaN  
1  24.55   34.06     3.68     

In [9]:
# BASIC INFORMATION
print(df.info())
print(df.describe())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29531 entries, 0 to 29530
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        29531 non-null  object 
 1   Date        29531 non-null  object 
 2   PM2.5       24933 non-null  float64
 3   PM10        18391 non-null  float64
 4   NO          25949 non-null  float64
 5   NO2         25946 non-null  float64
 6   NOx         25346 non-null  float64
 7   NH3         19203 non-null  float64
 8   CO          27472 non-null  float64
 9   SO2         25677 non-null  float64
 10  O3          25509 non-null  float64
 11  Benzene     23908 non-null  float64
 12  Toluene     21490 non-null  float64
 13  Xylene      11422 non-null  float64
 14  AQI         24850 non-null  float64
 15  AQI_Bucket  24850 non-null  object 
dtypes: float64(13), object(3)
memory usage: 3.6+ MB
None
              PM2.5          PM10            NO           NO2           NOx  \
count  24933.000